<a href="https://colab.research.google.com/github/cuie23/learning-pytorch/blob/main/ViT_paper_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following procedure from https://github.com/mrdbourke/pytorch-deep-learning/blob/main/08_pytorch_paper_replicating.ipynb

Replicating https://arxiv.org/pdf/2010.11929

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch.utils.data import DataLoader

from torch import nn
from torchvision import transforms, datasets
from torchvision.transforms import ToTensor

!pip install torchinfo
from torchinfo import summary

!git clone https://github.com/mrdbourke/pytorch-deep-learning
!mv pytorch-deep-learning/going_modular .
!mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
!rm -rf pytorch-deep-learning
from going_modular.going_modular import data_setup, engine
from helper_functions import download_data

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 4356 (delta 154), reused 120 (delta 120), pack-reused 4171 (from 3)
Receiving objects: 100% (4356/4356), 654.37 MiB | 41.38 MiB/s, done.
Resolving deltas: 100% (2584/2584), done.
Updating files: 100% (248/248), done.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

Set up data/dataloaders

In [4]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
train_dir = image_path / "train"
test_dir = image_path / "test"

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


In [ ]:
train_data = datasets.FashionMNIST(
    root="data",           # where to download data to?
    train=True,            # get training data
    download=True,         # download data if it doesn't exist on disk
    transform=ToTensor()   # images come as PIL format, turn into Torch tensors
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

BATCH_SIZE = 32

train_dataloader = DataLoader(
    train_data,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False     # don't necessarily have to shuffle the testing data
)

100%|██████████| 26.4M/26.4M [00:00<00:00, 63.1MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 71.6MB/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4.42M/4.42M [00:00<00:00, 37.7MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 15.8MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



NameError: name 'transforms' is not defined

In [16]:
IMG_SIZE = 224

transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])
transforms

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)

In [21]:
BATCH_SIZE = 16 # start with small batch size

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=transforms,
    batch_size=BATCH_SIZE
)

train_dataloader, test_dataloader, class_names


(<torch.utils.data.dataloader.DataLoader at 0x796ef37aa860>,
 ['pizza', 'steak', 'sushi'])

In [ ]:
image_batch, label_batch = next(iter(train_dataloader))
image, label = image_batch[0], label_batch[0]
image.shape, label

(torch.Size([1, 28, 28]), tensor(4))

## Rebuilding model outlined in Figure 1
1. Patch + position embedding
  * Partition image into subsections, turn into sequence of "patches"
2. Embed patches (as a vector)
3. Layer Normalization
  * Regularization
4. Multi-Head Attention
  * Multiple attention vectors computed in parallel and combined
5. MLP (Multi-Layer Perceptron)
  * Typical Dense NN
6. MLP Head
  * "Output Layer", converts learned features to output vector

## Math
* $z_0 = [x_{class}\ ;\ x^1_pE\ ;\ x^2_pE\ ;\ ...\ ;\ x^N_pE] + E_{pos}, E\in\mathbb{R}^{(P^2\cdot C)\times D}, E_{pos}\in \mathbb{R}^{(N+1)\times D}$
  * class token, patch embedding, position embedding
  * $z_0=$  output of initial layer
  * $E,\ E_{pos}=$embedding/position embedding
  * $C = $ number of channels
  * $D=$ hidden layer size
  * $P=$ side length of patch
  * $N=HW/P^2=$ number of patches
* $z'_\ell=MSA(LN(z_{\ell-1}))+z_{\ell-1}$
  * multi-head attention
  * MSA layer wrapping Layer Norm
  * $z'_\ell=$ intermediary output of $\ell^{th}$ layer
* $z_\ell=MLP(LN(z'_\ell))+z'_\ell$
  * multi-layer perceptron
  * MLP wrapping Layer Norm
  * $z_\ell=$ output of $\ell^{th}$ layer
* $y=LN(z^0_L)$
  * Linear dense output layer

In [ ]:
# Create variables
H = 224
W = 224
C = 3
P = 16
D = 768

N = H*W / (P**2)

# Embedding layer, create patches of size P
# Output shape of
conv2d = nn.Conv2d(in_channels=C, out_channels=D, kernel_size=P, stride=P)

# Flatten layer, only flatten feature map
flatten = nn.Flatten(start_dim = 2, end_dim = 3)

## Patch Embedding

In [ ]:
# See shape of embedding
# (batch_size, embedding_dim, feature_map_height, feature_map_width)
# feature_map_width = W / P
test_output = conv2d(image.unsqueeze(dim=0))
test_output.shape

torch.Size([1, 768, 14, 14])

In [ ]:
flatten(conv2d(image.unsqueeze(dim=0))).shape

torch.Size([1, 768, 196])

In [22]:
class PatchEmbedding(nn.Module):
    def __init__(self,
                 in_channels:int=3,
                 patch_size:int=16,
                 embedding_dim:int=768):
        super().__init__()

        self.P = patch_size

        # create patches
        self.patcher = nn.Conv2d(in_channels=in_channels,
                                 out_channels=embedding_dim,
                                 kernel_size=self.P,
                                 stride=self.P,
                                 padding=0)

        # flatten
        self.flatten = nn.Flatten(start_dim=2, # only flatten feature map dimensions
                                  end_dim=3)

    def forward(self, x):
        # assertion to check that inputs are the correct shape
        image_resolution = x.shape[-1]
        assert image_resolution % self.P == 0

        x_patched = self.patcher(x)
        x_flattened = self.flatten(x_patched)

        return x_flattened.permute(0, 2, 1) # adjust shape so final dim is correct

In [ ]:
embed = PatchEmbedding()
patch_embedded = embed(image.unsqueeze(dim=0))

patch_embedded.shape

torch.Size([1, 196, 768])

In [ ]:
H = 224
W = 224
C = 3
P = 16
D = 768

N = int(H*W / (P**2))
print(N)

batch_size = patch_embedded.shape[0]
embedding_dimension = patch_embedded.shape[2]

# Class token is random vector of length embedding_dimension
class_token = nn.Parameter(torch.randn(batch_size, 1, embedding_dimension))

embedded_with_token = torch.cat((class_token, patch_embedded), dim=1)
print(embedded_with_token.shape)


# add position embedding
number_of_patches = int((H * W) / P**2)

# Get embedding dimension
embedding_dimension = embedded_with_token.shape[2]

# Create the learnable 1D position embedding
position_embedding = nn.Parameter(torch.randn(batch_size,
                                             N+1,
                                             embedding_dimension))

print(position_embedding.shape)

patch_position_embedded = embedded_with_token + position_embedding
print(patch_position_embedded.shape)

196
torch.Size([1, 197, 768])
torch.Size([1, 197, 768])
torch.Size([1, 197, 768])


## MSA

In [23]:
class MultiheadSelfAttention(nn.Module):
  def __init__(self,
               embedding_dim: int=768,
               num_heads:int=12,
               dropout=0): # paper doesn't indicate any dropout
     super().__init__()

     # Use built in pytorch laeyrs
     self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

     self.multi_head_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                  num_heads=num_heads,
                                                  dropout=dropout,
                                                  batch_first=True)

  def forward(self, x):
    x = self.layer_norm(x)
    x, _ = self.multi_head_attn(query=x, # query embeddings
                                key=x,
                                value=x,
                                need_weights=False)
    return x

In [ ]:
multihead_self_attention_block = MultiheadSelfAttention(embedding_dim=768,
                                                             num_heads=12)
after_msa = multihead_self_attention_block(patch_position_embedded)
patch_position_embedded.shape, after_msa.shape

(torch.Size([1, 197, 768]), torch.Size([1, 197, 768]))

## MLP
* "two layers with a GELU non-linearity"
* "Dropout, when used, is applied after
every dense layer except for the the qkv-projections and directly after adding positional- to patch
embeddings"
* Structure is:
  1. layer norm
  2. linear
  3. GELU
  4. dropout
  5. linear
  6. dropout

In [24]:
class MLP(nn.Module):
  def __init__(self,
               embedding_dim:int=768,
               mlp_size: int=3072,
               dropout:float=0.1):
    super().__init__()

    self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

    self.mlp = nn.Sequential(
            nn.Linear(in_features=embedding_dim,
                      out_features=mlp_size),
            nn.GELU(),
            nn.Dropout(p=dropout), # probability of dropping an edge, used after every dense layer
            nn.Linear(in_features=mlp_size,
                      out_features=embedding_dim),
            nn.Dropout(p=dropout)
        )
  def forward(self, x):
        x = self.layer_norm(x)
        x = self.mlp(x)
        return x

In [ ]:
mlp_block = MLP(embedding_dim=768,
                     mlp_size=3072,
                     dropout=0.1)

after_mlp = mlp_block(after_msa)
after_msa.shape,after_mlp.shape

(torch.Size([1, 197, 768]), torch.Size([1, 197, 768]))

## Transformer Encoder
"The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded selfattention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before every block, and residual connections after every block (Wang et al., 2019; Baevski & Auli, 2019)."
  * Residual connection - input is added to output

In [25]:
class TransformerEncoder(nn.Module):
  def __init__(self,
               embedding_dim:int=768,
               num_heads:int=12,
               mlp_size:int=3072,
               mlp_dropout:float=0.1,
               attn_dropout:float=0.1):
    super().__init__()

    # Use previously created MSA/MLP classes
    self.msa = MultiheadSelfAttention(embedding_dim=embedding_dim,
                                      num_heads=num_heads,
                                      dropout=attn_dropout)
    self.mlp = MLP(embedding_dim=embedding_dim,
                   mlp_size=mlp_size,
                   dropout=mlp_dropout)

  def forward(self, x):
    x = self.msa(x) + x # adding input (residual connection)
    x = self.mlp(x) + x

    return x

In [ ]:
transformer_encoder = TransformerEncoder()
print(transformer_encoder)


TransformerEncoder(
  (msa): MultiheadSelfAttention(
    (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (multi_head_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
    )
  )
  (mlp): MLP(
    (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU(approximate='none')
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=3072, out_features=768, bias=True)
      (4): Dropout(p=0.1, inplace=False)
    )
  )
)


Alternatively use pytorch's built in transformer layer

In [ ]:
torch_transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=768,
                                                             nhead=12,
                                                             dim_feedforward=3072,    # MLP size
                                                             dropout=0.1,             # Amount of dropout for dense layers
                                                             activation="gelu",       # GELU non-linear activation
                                                             batch_first=True,
                                                             norm_first=True)         # Normalize before MSA/MLP layers
torch_transformer_encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (linear1): Linear(in_features=768, out_features=3072, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=3072, out_features=768, bias=True)
  (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Final Model

In [26]:
class ViT(nn.Module):
  # hyperparams from Tables 1 and 3
  def __init__(self,
              img_size:int=224,       # H/W
              in_channels:int=3,      # C
              patch_size:int=16,      # P
              num_transformer_layers:int=12,
              embedding_dim:int=768,  # D
              mlp_size:int=3072,
              num_heads:int=12,
              attn_dropout:float=0,
              mlp_dropout:float=0.1,
              embedding_dropout:float=0.1,
              num_classes:int=1000):
    super().__init__()

    assert img_size % patch_size == 0  # check divisibility

    # Constants
    self.num_patches = int(img_size**2 / patch_size**2)   # N
    self.class_embedding = nn.Parameter(torch.randn(1, 1, embedding_dim), requires_grad=True)   # x_class
    self.position_embedding = nn.Parameter(torch.randn(1, self.num_patches+1, embedding_dim), requires_grad=True)   # E_pos

    # Layers
    self.embedding_dropout = nn.Dropout(p=embedding_dropout)
    self.patch_embedding = PatchEmbedding(in_channels=in_channels,
                                          patch_size=patch_size,
                                          embedding_dim=embedding_dim)
    self.transformer_encoder = nn.Sequential(*[TransformerEncoder(embedding_dim=embedding_dim,      # * means "for all", basically just follow loop
                                                                  num_heads=num_heads,
                                                                  mlp_size=mlp_size,
                                                                  mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])

    self.classifier = nn.Sequential(  # aka "MLP head"
            nn.LayerNorm(normalized_shape=embedding_dim),
            nn.Linear(in_features=embedding_dim,
                      out_features=num_classes)
        )

  def forward(self, x):

        batch_size = x.shape[0]

        class_token = self.class_embedding.expand(batch_size, -1, -1)
        x = self.patch_embedding(x)
        x = torch.cat((class_token, x), dim=1)  # concat class token
        x = self.position_embedding + x
        x = self.embedding_dropout(x)
        x = self.transformer_encoder(x)
        x = self.classifier(x[:, 0])

        return x

## Testing

In [27]:
vit = ViT(#img_size=28,
          #in_channels=1,
          #patch_size=2,
          num_classes=len(class_names))

In [28]:
optimizer = torch.optim.Adam(params=vit.parameters(), # default value from paper
                             lr=3e-3,
                             betas=(0.9, 0.999),
                             weight_decay=0.3)

loss_fn = torch.nn.CrossEntropyLoss()

set_seeds()

results = engine.train(model=vit,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device)



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.4127 | train_acc: 0.2500 | test_loss: 1.7502 | test_acc: 0.2375
Epoch: 2 | train_loss: 1.2267 | train_acc: 0.2917 | test_loss: 1.1630 | test_acc: 0.4500
Epoch: 3 | train_loss: 1.1501 | train_acc: 0.3375 | test_loss: 1.1169 | test_acc: 0.4500
Epoch: 4 | train_loss: 1.1399 | train_acc: 0.3292 | test_loss: 1.2340 | test_acc: 0.3125
Epoch: 5 | train_loss: 1.1294 | train_acc: 0.4167 | test_loss: 1.4692 | test_acc: 0.2375
Epoch: 6 | train_loss: 1.3194 | train_acc: 0.3167 | test_loss: 1.0929 | test_acc: 0.4500
Epoch: 7 | train_loss: 1.2149 | train_acc: 0.3250 | test_loss: 1.1783 | test_acc: 0.4500
Epoch: 8 | train_loss: 1.2402 | train_acc: 0.2750 | test_loss: 1.1911 | test_acc: 0.2375
Epoch: 9 | train_loss: 1.2036 | train_acc: 0.3208 | test_loss: 1.0744 | test_acc: 0.4500
Epoch: 10 | train_loss: 1.1401 | train_acc: 0.2875 | test_loss: 1.1190 | test_acc: 0.3125
Epoch: 11 | train_loss: 1.1558 | train_acc: 0.2875 | test_loss: 1.1736 | test_acc: 0.2375
Epoch: 12 | train_l

In [29]:
from sklearn.metrics import classification_report

all_X, all_y = [], []
for X, y in test_dataloader:
    X, y = X.to(device), y.to(device)
    all_X.append(X)  # Append each batch
    all_y.append(y)

X_test = torch.cat(all_X, dim=0)
y_test = torch.cat(all_y, dim=0)

final_preds = vit(X_test)

print(classification_report(y_test.cpu().numpy(), final_preds.argmax(dim = 1).cpu().numpy()))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50        25
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00        31

    accuracy                           0.33        75
   macro avg       0.11      0.33      0.17        75
weighted avg       0.11      0.33      0.17        75



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!nvidia-smi

Tue Dec 31 02:36:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |  15073MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
print(torch.cuda.memory_summary(device=torch.device('cuda')))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  14732 MiB |  14751 MiB |   4680 GiB |   4666 GiB |
|       from large pool |  14708 MiB |  14726 MiB |   4648 GiB |   4634 GiB |
|       from small pool |     24 MiB |     51 MiB |     32 GiB |     32 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  14732 MiB |  14751 MiB |   4680 GiB |   4666 GiB |
|       from large pool |  14708 MiB |  14726 MiB |   4648 GiB |